In [2]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_api as sa
import numpy as np
import pandas as pd
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select

# Para trabajar con Mongo
# ------------------------------------------------------------------------------
import pymongo

In [3]:
# lo primero que tenemos que hacer es conectarnos con Mongo
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']

In [5]:
df = pd.read_csv("../data/api_generos.csv")

In [11]:
#Abrimos el Chrome
driver = webdriver.Chrome()

# Maximizar la ventana del navegador
driver.maximize_window()

lista_info_pelis_final = []

# Navegar a la página web de IMDB 
driver.get("https://www.imdb.com")

#Introducimos id_peli y se busca
for id_peli in tqdm(df.loc[::10,'id']): #para sacar el 10% de las peliculas
    if peliculas.find_one({'id_IMDB': id_peli}):
        continue
    dicc = {}
    dicc['id_IMDB'] = id_peli
    #Quitamos lo de la cuenta
    try:
        cuenta=driver.find_element('css selector','#imdbHeader > div.ipc-page-content-container.ipc-page-content-container--center.navbar__inner > div.nav__userMenu.navbar__user.sc-jfTVlA.esWGB > div > div > div > div > div.sc-cOxWqc.bNDQpU > button > svg').click()
        #id_peli='tt0075259' #Ejemplo, luego habrá que hacer bucles for :( 

    except:
        pass
    
    try:
        busqueda_peli = driver.find_element('css selector', '#suggestion-search').send_keys(id_peli, Keys.ENTER)
    except:
        driver.close()
        #Abrimos el Chrome
        driver = webdriver.Chrome()

        # Maximizar la ventana del navegador
        driver.maximize_window()

        lista_info_pelis_final = []

        # Navegar a la página web de IMDB 
        driver.get("https://www.imdb.com")
        busqueda_peli = driver.find_element('css selector', '#suggestion-search').send_keys(id_peli, Keys.ENTER)
    try:
        dicc['puntuacion'] = float(driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div.sc-3a4309f8-0.jJkxPn.sc-70a366cc-1.kUfGfN > div > div:nth-child(1) > a > span > div > div.sc-d541859f-0.hNIoIx > div.sc-d541859f-2.kxphVf > span.sc-d541859f-1.imUuxf').text.replace(",","."))
    except:
        dicc['puntuacion'] = np.nan
    #driver.implicitly_wait(2)
    directores = []
    guionistas = []

    for i in range(1,5):
        try:

            directores.append(driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
        except:
            pass

        try:
            guionistas.append(driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
        except:
            pass

        dicc['direccion'] = directores

        dicc['guion'] = guionistas


    
    
    try:
        dicc['duracion'] = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(3)').text.replace('h','').replace('min','').split()
        if len(dicc['duracion']) == 1:
            try:
                dicc['duracion'] = int(dicc['duracion'][0])
            except:
                dicc['duracion'] = np.nan
        else:
            try:
                dicc['duracion'] = int(dicc['duracion'][0])*60 + int(dicc['duracion'][1])
            except:
                dicc['duracion'] = np.nan
    except:
        
        try:
            dicc['duracion']  =driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-9a2a0028-3.bwWOiy > div.sc-70a366cc-0.bxYZmb > ul > li:nth-child(2)').text.replace('h','').replace('min','').split()
            if len(dicc['duracion']) == 1:
                try:
                    dicc['duracion'] = int(dicc['duracion'][0])
                except:
                    dicc['duracion'] = np.nan
            else:
                try:
                    dicc['duracion'] = int(dicc['duracion'][0])*60 + int(dicc['duracion'][1])
                except:
                    dicc['duracion'] = np.nan
        except:
            dicc['duracion'] = np.nan
    
    try:
        dicc['argumento'] = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
        if dicc['argumento'] == '':
            dicc['argumento'] = np.nan
    except:
        dicc['argumento'] = np.nan
        # Insertar el documento

    peliculas.insert_one(dicc)


driver.close()

100%|██████████| 31458/31458 [1:19:45<00:00,  6.57it/s]


In [17]:
driver = webdriver.Chrome()

driver.maximize_window()

driver.get("https://www.imdb.com/")

for peli in tqdm(list(peliculas.find())):
    if 'actores' in peli.keys():
        continue
    
    try:
        driver.find_element("css selector", "#suggestion-search").send_keys(peli['id_IMDB'], Keys.ENTER)
    except:
        driver.close()
        #Abrimos el Chrome
        driver = webdriver.Chrome()

        # Maximizar la ventana del navegador
        driver.maximize_window()

        # Navegar a la página web de IMDB 
        driver.get("https://www.imdb.com")
        driver.find_element("css selector", "#suggestion-search").send_keys(peli['id_IMDB'], Keys.ENTER)

    lista_actores = []
    for i in range(1, 11):
        try:
            lista_actores.append(driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-c0933c3e-1.fQDAuW.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-cd7dc4b7-0.ycheS.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-cd7dc4b7-7.vCane > a").text)
                                                                       #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-c0933c3e-1.fQDAuW.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-cd7dc4b7-0.ycheS.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-cd7dc4b7-7.vCane > a
        except:
            pass

    peliculas.update_one({'id_IMDB':peli['id_IMDB']},{'$set': {'actores': lista_actores}})


driver.close()


100%|██████████| 31487/31487 [58:16<00:00,  9.00it/s]  
